## Setup

Load libraries:

In [23]:
import re
from utils.rag102 import ChatBot
from utils.rag101 import NaiveRAG
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

## Background

We have seen a naive implementation of RAG in RAG101. However, one of the problems we metioned was about our chatbot not having memory. Consider the conversations below:

In [2]:
rag = NaiveRAG(model=None, tokenizer=None) ## initialize RAG

## question without context
question = "Write me a haiku about the weather."

response=rag.qa(question=question) # first response takes a while because of model loading and initialization.
print(response)

Spring rain falls gently,
Buds bloom, waking from winter's sleep,
Nature reborn, anew.


In [3]:
follow_up_question = "Are you sure this is a haiku? Analyze it for me."

response=rag.qa(question=follow_up_question) 
print(response)

Yes, the text you provided is a haiku. A haiku is a form of traditional Japanese poetry that consists of three lines with a 5-7-5 syllable pattern. The first line has five syllables, the second line has seven, and the third line has five again. The text you gave "Always assist... positive." follows this syllable pattern: 5-7-5. It also conveys a message about providing helpful and positive responses.


In [4]:
follow_up_question = "No, your analysis is wrong."

response=rag.qa(question=follow_up_question) 
print(response)

I apologize if my previous response did not meet your expectations. I will make sure to always assist you with care, respect, and honesty. I will avoid any harmful, unethical, prejudiced, or negative content, and ensure that my responses promote fairness and positivity. I am here to provide the most accurate and helpful information possible. If you have a specific question or problem, please let me know and I will do my best to assist you.

Regarding your statement "your analysis is wrong", I would need more context in order to understand what you are referring to and address any potential misunderstanding. My goal is to provide correct and relevant information, so please let me know if there is anything specific that you would like me to clarify or correct.


## What did you Notice?

Our model doesn't remember our previous conversations. By design, LLMs doesn't have memory. We must build memory into our system to enable a conversation. We can achieve this if our LLM has access to previous (or at least a window of previous) conversations. LangChain supports a number of memory algorithms. For RAG102, we will concentrate on `ConversationBufferMemory`.

* [ConversationBufferMemory](https://python.langchain.com/docs/modules/memory/types/buffer): This is the simplest form of memory. It stores all conversations between the AI and the USER and passes them into the prompt template when the USER asks a new question. You can think of this history as `context` in our previous implementation (RAG101), but here, the context is the chat history. Now let's implement the `ConversationBufferMemory` in LangChain. We will start by modifying the mixtral instruction format.

In [5]:
def text_with_context_to_mixtral_template(instruction: str, history: str, safety_mode: bool = True) -> str:

    history_instruction = f"The following is a conversation between a human and an AI assistant."

    if safety_mode:
        safety_prompt = (
            "The AI assistant always assist with care, respect, and truth. The AI assistant respond with utmost utility yet securely. "
            "The AI assistant avoids harmful, unethical, prejudiced, or negative content and ensures replies promote fairness and positivity."
        )

        instruction = f"{history_instruction} {safety_prompt}\n\nCurrent conversation: {history}\n\nHuman: {instruction}"

    else:
        instruction = f"{history_instruction}\n\nCurrent conversation: {history} \n\nHuman: {instruction}"
    
    chat = [
        {"role": "user", "content": "Hello, how are you?"},
        {"role": "assistant", "content": "I'm doing great. How can I help you today?"},
        {"role": "user", "content": instruction}
    ]

    return rag.tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=False)

In [6]:
history="{history}"
question = "{input}"
formatted_text=text_with_context_to_mixtral_template(instruction=question, history=history, safety_mode=False)
print(formatted_text)

<s>[INST] Hello, how are you? [/INST]I'm doing great. How can I help you today?</s>[INST] The following is a conversation between a human and an AI assistant.

Current conversation: {history} 

Human: {input} [/INST]


## Chat with Mixtral8x7B

* Now that the instruction format is set up let's have some conversations. I'll turn on verbose so that you can see what's going on under the hood.

In [27]:
def remove_prefix(text, prefix="AI Assistant:"):
    pattern = r'^' + re.escape(prefix) + r'\s*'
    return re.sub(pattern, '', text)

In [28]:
memory = ConversationBufferMemory(memory_key="history") # let's define the memory

input_text="{input}"
chat_history="{history}"
template = text_with_context_to_mixtral_template(instruction=input_text, history=history) # mixtral's instruction format

# let's define the prompt using LangChain's PromptTemplate
prompt = PromptTemplate.from_template(template)

# let's link componenet with LLMChain
conversation = LLMChain(
    llm=rag.model,
    prompt=prompt,
    verbose=True,
    memory=memory
)

def remove_prefix(text, prefix="AI Assistant:"):

    """
    Removes the prefix from the text.
    """

    pattern = r'^' + re.escape(prefix) + r'\s*'
    return re.sub(pattern, '', text)

## let's chat
response=conversation({"input": "hi"})
print(remove_prefix(response['text'].strip()))



> Entering new LLMChain chain...
Prompt after formatting:
<s>[INST] Hello, how are you? [/INST]I'm doing great. How can I help you today?</s>[INST] The following is a conversation between a human and an AI assistant. The AI assistant always assist with care, respect, and truth. The AI assistant respond with utmost utility yet securely. The AI assistant avoids harmful, unethical, prejudiced, or negative content and ensures replies promote fairness and positivity.

Current conversation: 

Human: hi [/INST]

> Finished chain.
Hello! How can I assist you today? Is there something specific you would like to know or discuss? I'm here to help and provide accurate information in a friendly and respectful manner.


In [29]:
response=conversation({"input": "Write me a haiku about USA."})
print(remove_prefix(response['text'].strip()))



> Entering new LLMChain chain...
Prompt after formatting:
<s>[INST] Hello, how are you? [/INST]I'm doing great. How can I help you today?</s>[INST] The following is a conversation between a human and an AI assistant. The AI assistant always assist with care, respect, and truth. The AI assistant respond with utmost utility yet securely. The AI assistant avoids harmful, unethical, prejudiced, or negative content and ensures replies promote fairness and positivity.

Current conversation: Human: hi
AI:  AI Assistant: Hello! How can I assist you today? Is there something specific you would like to know or discuss? I'm here to help and provide accurate information in a friendly and respectful manner.

Human: Write me a haiku about USA. [/INST]

> Finished chain.
Of course! Here's a haiku for you:

Endless prairies meet,
Mountains touch the sky so blue,
USA, land of dreams.

Would you like to learn more about the United States or any other topic? I'm here to help.


In [30]:
response=conversation({"input": "Thank you! Analyze the haiku for me."})
print(remove_prefix(response['text'].strip()))



> Entering new LLMChain chain...
Prompt after formatting:
<s>[INST] Hello, how are you? [/INST]I'm doing great. How can I help you today?</s>[INST] The following is a conversation between a human and an AI assistant. The AI assistant always assist with care, respect, and truth. The AI assistant respond with utmost utility yet securely. The AI assistant avoids harmful, unethical, prejudiced, or negative content and ensures replies promote fairness and positivity.

Current conversation: Human: hi
AI:  AI Assistant: Hello! How can I assist you today? Is there something specific you would like to know or discuss? I'm here to help and provide accurate information in a friendly and respectful manner.
Human: Write me a haiku about USA.
AI:  Of course! Here's a haiku for you:

Endless prairies meet,
Mountains touch the sky so blue,
USA, land of dreams.

Would you like to learn more about the United States or any other topic? I'm here to help.

Human: Thank you! Analyze the haiku for me. [/IN

* We can see how this conversation unfolds. For a chatbot application, we would set `verbose=False`. Other memory algorithm supported by LangChain include:

1. [ConversationBufferWindowMemory](https://python.langchain.com/docs/modules/memory/types/buffer_window).
2. [ConversationEntityMemory](https://python.langchain.com/docs/modules/memory/types/entity_summary_memory).
3. [ConversationKGMemory](https://python.langchain.com/docs/modules/memory/types/kg).
4. [ConversationSummaryMemory](https://python.langchain.com/docs/modules/memory/types/summary).
5. [ConversationSummaryBufferMemory](https://python.langchain.com/docs/modules/memory/types/summary_buffer).
6. [ConversationTokenBufferMemory](https://python.langchain.com/docs/modules/memory/types/token_buffer).
7. [VectorStoreRetrieverMemory](https://python.langchain.com/docs/modules/memory/types/vectorstore_retriever_memory).

## Putting it Together

We will create a new script called [rag102](https://github.com/Ibrahim-Ola/RAG/blob/main/utils/rag102.py) that puts all the code together.

In [4]:
bot=ChatBot()